In [3]:
import numpy as np

import qiskit
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, assemble
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import PulseSimulator, Aer, QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.pulse import PulseSystemModel
from qiskit.test.mock import FakeArmonk
from qiskit.tools import job_monitor
from src.helper import qutip_ham_converter
from src.qoc_instruction_schedule_map import QOCInstructionScheduleMap
from src.QutipOptimizer import QutipOptimizer

IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
real_armonk_backend = provider.get_backend('ibmq_armonk')

In [4]:
# real_armonk_backend.default

In [5]:
fake_armonk_backend=FakeArmonk()
which_armonk = 'fake'
# which_armonk = 'real'
# def_x = True
# if which_armonk=='real':
# fake_armonk_backend=real_armonk_backend
# doesn't work with name()'

In [6]:
# convert hamiltonian from backend to qutip nbformat
# hamiltonian = qutip_ham_converter(fake_armonk_backend)

# Instantiate grape optimizer with this hamiltonian
grape_optimizer = QutipOptimizer(fake_armonk_backend)

# Create new QOCInstructionScheduleMap with this optimizer
grape_inst_map = QOCInstructionScheduleMap(grape_optimizer)

In [7]:
# add measurement map to grape_inst_map
# this isn't long term sol, for measurement probably just use getter or something in instantiation
grape_inst_map._map['measure'] = fake_armonk_backend.defaults().instruction_schedule_map._map['measure']
# grape_inst_map._map['MEAS'] = fake_armonk_backend.defaults().instruction_schedule_map._map['MEAS']
grape_inst_map._qubit_instructions = fake_armonk_backend.defaults().instruction_schedule_map._qubit_instructions
# grape_inst_map = fake_armonk_backend.defaults().instruction_schedule_map

In [8]:
# create a random gate for testing
rand_gate = UnitaryGate(qiskit.quantum_info.random_unitary(2,2))
rand_gate.to_matrix()

array([[ 0.10032203+0.95502284j, -0.26220766-0.09546728j],
       [-0.21919403-0.1726871j , -0.86480991+0.41741702j]])

In [9]:
# Create circuit
q = QuantumRegister(1)
c = ClassicalRegister(1)
circ = qiskit.QuantumCircuit(q, c)

# Add the X gate
# circ.x(q)
# circ.x(q)
# # circ.h(q)
circ.u3(np.pi,0,np.pi,q)

# circ.append(rand_gate, q)

# Add the measurement pulse
circ.measure([0], [0])

In [10]:
# set the drive strength
omegad0 = 31919806.545849085
# omegad0/ 2 / np.pi

In [11]:
getattr(fake_armonk_backend.configuration(), 'hamiltonian')['vars']['omegad0'] = omegad0


# set the qubit frequency
freq_est = 4.97445401836326e9
fake_armonk_backend.configuration().qubit_freq_est=[freq_est]
getattr(fake_armonk_backend.configuration(), 'hamiltonian')['vars']['wq0'] = 2*np.pi*freq_est

In [12]:
# generate model from backend
armonk_model = PulseSystemModel.from_backend(fake_armonk_backend)

In [13]:
# construct the schedule from the circuit using the grape instruction_schedule_map
grape_schedule = qiskit.schedule(circ, inst_map = grape_inst_map,
                        meas_map = fake_armonk_backend.configuration().meas_map)

INFO:qutip.control.dynamics:Setting memory optimisations for level 0
INFO:qutip.control.dynamics:Internal operator data type choosen to be <class 'numpy.ndarray'>
INFO:qutip.control.dynamics:phased dynamics generator caching True
INFO:qutip.control.dynamics:propagator gradient caching True
INFO:qutip.control.dynamics:eigenvector adjoint caching True
INFO:qutip.control.dynamics:use sparse eigen decomp False
INFO:qutip.control.pulseoptim:System configuration:
Drift Hamiltonian:
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 0.]]
Control 1 Hamiltonian:
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[   0.         3191.98065458]
 [3191.98065458    0.        ]]
Control 2 Hamiltonian:
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.   +0.j         0.-3191.98065458j]
 [0.+3191.98065458j 0.   +0.j        ]]
Initial state / operator:
Quantum object: 

gate hit


In [14]:
# assemble qobj for job submission
backend_sim = PulseSimulator(configuration=fake_armonk_backend.configuration)

grape_qobj = assemble(grape_schedule,
                      backend=backend_sim,
                      qubit_lo_freq=[freq_est],
                      meas_level=2,
                      meas_return='single',
                      shots=1024)

In [15]:
sim_result = backend_sim.run(grape_qobj, armonk_model).result()
sim_result.get_counts()

{'1': 1024}

In [16]:
noise_model = NoiseModel()
noise_model.add_basis_gates(['unitary'])
ideal_job = qiskit.execute(circ, QasmSimulator(),
                           basis_gates=noise_model.basis_gates)
ideal_job.result().get_counts()

{'1': 1024}

In [17]:
# now we run it on the real armonk
armonk_qobj = assemble(grape_schedule,
                      backend=real_armonk_backend,
                      qubit_lo_freq=[freq_est],
                      meas_level=2,
                      meas_return='single',
                      shots=1024)
# real_job = real_armonk_backend.run(armonk_qobj)
job_id = real_job._job_id

NameError: name 'real_armonk_backend' is not defined

In [ ]:
job = real_armonk_backend.retrieve_job(job_id)
job_monitor(job)
job.result().get_counts()

#%

print('ideal results: {}'.format(ideal_job.result().get_counts()))
print('simulated results: {}'.format(sim_result.get_counts()))
# print('real results: {}'.format(real_job.result().get_counts()))
print('Grape run on {} backend'.format(which_armonk))
print('Used grape gate: {}'.format(not def_x))

In [ ]:
# def x gate on real armonk: 0:67, 1:957
# grape x gate on real armonk: 0:212, 1:812
# see https://qiskit.org/documentation/locale/de_DE/tutorials/pulse/5_pulse_simulator_backend_model.html